# Tensorflow的可视化工具Tensorboard的初步使用

当使用Tensorflow训练大量深层的神经网络时，我们希望去跟踪神经网络的整个训练过程中的信息，比如迭代的过程中每一层参数是如何变化与分布的，比如每次循环参数更新后模型在测试集与训练集上的准确率是如何的，比如损失值的变化情况，等等。如果能在训练的过程中将一些信息加以记录并可视化得表现出来，是不是对我们探索模型有更深的帮助与理解呢？

Tensorflow官方推出了可视化工具Tensorboard，可以帮助我们实现以上功能，它可以将模型训练过程中的各种数据汇总起来存在自定义的路径与日志文件中，然后在指定的web端可视化地展现这些信息。

## 1. Tensorboard介绍

### 1.1 Tensorboard的数据形式

Tensorboard可以记录与展示以下数据形式： 
* 标量Scalars 
* 图片Images 
* 音频Audio 
* 计算图Graph 
* 数据分布Distribution 
* 直方图Histograms 
* 嵌入向量Embeddings

### 1.2 Tensorboard的可视化过程

1. 首先肯定是先建立一个graph,你想从这个graph中获取某些数据的信息
2. 确定要在graph中的哪些节点放置summary operations以记录信息 
    * 使用tf.summary.scalar记录标量 
    * 使用tf.summary.histogram记录数据的直方图 
    * 使用tf.summary.distribution记录数据的分布图 
    * 使用tf.summary.image记录图像数据 
3. 以上summary operations并不会主动去执行计算，除非你命令执行这些summary operations计算(通过session.run()), 或者它被其他的需要run的operation所依赖。而我们上一步创建的这些summary operations其实并不被其他节点依赖，因此，我们需要特地去运行所有的summary节点。但是，我们通常会设置很多这样的summary节点，要手动一个一个去启动自然是及其繁琐的，因此我们可以使用tf.summary.merge_all去将所有summary节点合并成一个节点，只要运行这个节点，就能产生所有我们之前设置的summary data。
4. 使用tf.summary.FileWriter将运行后输出的数据都保存到本地磁盘中
5. 在命令行输入运行tensorboard的指令，之后打开web端可查看可视化的结果

## 2. Tensorboard使用案例

我们使用最基础的识别手写字体的案例. 本案例中，我们不追求最优的模型，只是建立一个简单的神经网络，让大家了解如何使用Tensorboard。

### 2.1 导入包，定义超参数，载入数据


2.1.1. 首先还是导入需要的包：

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

2.1.2 定义固定的超参数,方便待使用时直接传入。

在这里我们不讨论如何选择超参数，而假设我们已经获得了最优的超参数，其中设置
* learning rate为0.001，
* dropout的保留节点比例为0.9，
* epoch为1000.

另外，还要设置两个路径，第一个是数据下载下来存放的地方，一个是summary输出保存的地方。

In [6]:
max_step = 1000  # 最大迭代次数
learning_rate = 0.001   # 学习率
dropout = 0.9   # dropout时随机保留神经元的比例

data_dir = 'tensorboard/data'   # 样本数据存储的路径
log_dir = 'tensorboard'    # 输出日志保存的路径

2.1.3 加载数据

下载数据是直接调用了tensorflow提供的函数read_data_sets,输入两个参数，第一个是下载到数据存储的路径，第二个one_hot表示是否要将类别标签进行独热编码。它首先回去找制定目录下有没有这个数据文件，没有的话才去下载，有的话就直接读取。所以第一次执行这个命令，速度会比较慢。

In [7]:
mnist = input_data.read_data_sets(data_dir,one_hot=True)

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorboard/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting tensorboard/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting tensorboard/data/t10k-images-idx3-ubyte.gz
Extracting tensorboard/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### 2.2 创建特征与标签的占位符，保存输入的图片数据到summary

2.2.1 创建tensorflow的默认会话：

In [8]:
tf.reset_default_graph()

2.2.2 创建输入数据的占位符，分别创建特征数据x，标签数据y_ 

在tf.placeholder()函数中传入了3个参数，第一个是定义数据类型为float32；第二个是数据的shape. 特征数据是大小784的向量，标签数据是大小为10的向量，None表示不定死大小，到时候可以传入任何数量的样本；第3个参数是这个占位符的名称。

In [9]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')

2.2.3 使用tf.summary.image保存图像信息 

特征数据其实就是图像的像素数据拉升成(flatten)一个`[1x784]`的向量，现在如果想在tensorboard上还原出输入的特征数据对应的图片，就需要将拉升的向量转变成`[28x28x1]`的原始像素了，于是可以用tf.reshape()直接重新调整特征数据的维度, 将输入的数据转换成`[28x28x1]`的shape，存储成另一个tensor，命名为image_shaped_input。 

为了能使图片在tensorbord上展示出来, 使用`tf.summary.image()`函数将图片数据汇总给tensorbord. tf.summary.image()中传入的第一个参数是命名，第二个是图片数据，第三个是最多展示的张数，此处为10张

In [10]:
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

### 2.3 创建初始化参数的方法，与参数信息汇总到summary的方法

2.3.1 在构建神经网络模型中，每一层中都需要去初始化参数w,b, 为了使代码方便管理，最好将初始化参数的过程封装成方法function. 创建初始化权重w的方法，生成大小等于传入的shape参数，标准差为0.1,正态分布的随机数，并且将它转换成tensorflow中的variable返回。

In [11]:
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

创建初始换偏执项b的方法，生成大小为传入参数shape的常数0.1，并将其转换成tensorflow的variable并返回

In [12]:
def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

2.3.2 在训练的过程在参数是不断地在改变和优化的，我们往往想知道每次迭代后参数都做了哪些变化，可以将参数的信息展现在tenorbord上，因此我们专门写一个方法来收录每次的参数信息。

In [13]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        # 计算参数的均值，并使用tf.summary.scaler记录
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)

        # 计算参数的标准差
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            # 使用tf.summary.scaler记录记录下标准差，最大值，最小值
            tf.summary.scalar('stddev', stddev)
            tf.summary.scalar('max', tf.reduce_max(var))
            tf.summary.scalar('min', tf.reduce_min(var))
            # 用直方图记录参数的分布
            tf.summary.histogram('histogram', var)

### 2.4 构建神经网络层

2.4.1 创建第一层隐藏层 

创建一个构建隐藏层的方法,输入的参数有： 
* `input_tensor`：特征数据 
* `input_dim`：输入数据的维度大小 
* `output_dim`：输出数据的维度大小(=隐层神经元个数） 
* `layer_name`：命名空间 
* `act=tf.nn.relu`：激活函数（默认是relu)

In [14]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # 设置命名空间
    with tf.name_scope(layer_name):
        # 调用之前的方法初始化权重w，并且调用参数信息的记录方法，记录w的信息
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        # 调用之前的方法初始化权重b，并且调用参数信息的记录方法，记录b的信息
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        # 执行wx+b的线性计算，并且用直方图记录下来
        with tf.name_scope('linear_compute'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('linear', preactivate)
        # 将线性输出经过激励函数，并将输出也用直方图记录下来
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)

      # 返回激励层的最终输出
    return activations

调用隐层创建函数创建一个隐藏层：输入的维度是特征的维度784，神经元个数是500，也就是输出的维度。

In [15]:
hidden1 = nn_layer(x, 784, 500, 'layer1')

2.4.2 创建一个dropout层，,随机关闭掉hidden1的一些神经元，并记录keep_prob

In [16]:
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

2.4.3 创建一个输出层，输入的维度是上一层的输出：500,输出的维度是分类的类别种类：10，激活函数设置为全等映射identity.（暂且先别使用softmax,会放在之后的损失函数中一起计算）

In [17]:
y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

### 2.5 创建损失函数

使用`tf.nn.softmax_cross_entropy_with_logits`来计算softmax并计算交叉熵损失,并且求均值作为最终的损失值。

In [18]:
with tf.name_scope('loss'):
    # 计算交叉熵损失（每个样本都会有一个损失）
    diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)
    with tf.name_scope('total'):
      # 计算所有样本交叉熵损失的均值
      cross_entropy = tf.reduce_mean(diff)

tf.summary.scalar('loss', cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'loss_1:0' shape=() dtype=string>

### 2.6 训练，并计算准确率

2.6.1 使用AdamOptimizer优化器训练模型，最小化交叉熵损失

In [19]:
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

2.6.2 计算准确率,并用`tf.summary.scalar()`记录准确率

In [20]:
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      # 分别将预测和真实的标签中取出最大值的索引，弱相同则返回1(true),不同则返回0(false)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    with tf.name_scope('accuracy'):
      # 求均值即为准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

### 2.7 合并summary operation, 运行初始化变量

将所有的summaries合并，并且将它们写到之前定义的log_dir路径

In [22]:
# summaries合并
merged = tf.summary.merge_all()

### 2.8 准备训练数据与测试数据，循环执行整个graph进行训练与评估

2.8.1 准备为训练与测试的输入数据. 

* 如果是`train==true`，就从mnist.train中获取一个batch样本，并且设置dropout值。
* 如果是`train==false`,则获取minist.test的测试数据，并且设置keep_prob为1，即保留所有神经元开启。

In [23]:
def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train:
        xs, ys = mnist.train.next_batch(100)
        k = dropout
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

2.8.2 训练模型

* 每隔10步，就进行一次summary merge, 并打印一次测试数据集的准确率(accuracy)，然后将测试数据集的各种summary信息写进日志中。 
* 每隔100步，记录原信息其他每一步时都记录下训练集的summary信息并写到日志中。

In [26]:
# 运行初始化所有变量
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 写到指定的磁盘路径中
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')

    for i in range(max_step):
        if i % 10 == 0:  # 记录测试集的summary与accuracy
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s: %s' % (i, acc))
        else:  # 记录训练集的summary
            if i % 100 == 99:  # Record execution stats
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                  feed_dict=feed_dict(True),
                                  options=run_options,
                                  run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:  # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)
    train_writer.close()
    test_writer.close()

Accuracy at step 0: 0.1311
Accuracy at step 10: 0.6706
Accuracy at step 20: 0.8277
Accuracy at step 30: 0.8661
Accuracy at step 40: 0.881
Accuracy at step 50: 0.8922
Accuracy at step 60: 0.9001
Accuracy at step 70: 0.9055
Accuracy at step 80: 0.9045
Accuracy at step 90: 0.9149
Adding run metadata for 99
Accuracy at step 100: 0.921
Accuracy at step 110: 0.9205
Accuracy at step 120: 0.9254
Accuracy at step 130: 0.9244
Accuracy at step 140: 0.9284
Accuracy at step 150: 0.9278
Accuracy at step 160: 0.9273
Accuracy at step 170: 0.9327
Accuracy at step 180: 0.935
Accuracy at step 190: 0.9363
Adding run metadata for 199
Accuracy at step 200: 0.9354
Accuracy at step 210: 0.9374
Accuracy at step 220: 0.934
Accuracy at step 230: 0.9358
Accuracy at step 240: 0.9359
Accuracy at step 250: 0.9387
Accuracy at step 260: 0.942
Accuracy at step 270: 0.9427
Accuracy at step 280: 0.9433
Accuracy at step 290: 0.9451
Adding run metadata for 299
Accuracy at step 300: 0.9455
Accuracy at step 310: 0.9459
Accur